In [6]:
import numpy as np
from scipy.linalg import svd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


### Question 2.1: Generate m Snapshot Data

In [7]:
# Parameters for snapshot generation
dt = 1e-2  # Time step
dx = 1e-3  # Spatial grid length
m = 20     # Number of snapshots

# Generate time and spatial grids
t = np.arange(0, m * dt, dt)  # Time grid from t1 to tm
x = np.arange(-1, 1 + dx, dx)  # Spatial grid

# Define the analytical solution for the Burgers equation
def analytical_solution(x, t, Re_fixed):
    t0 = np.exp(Re_fixed / 8)  # Parameter in the equation
    u = x / (t + 1) / (1 + np.sqrt((t + 1) / t0) * np.exp(Re_fixed * x**2 / (4 * (t + 1))))
    return u

# Generate snapshot data
Re_fixed = 100  # Reynolds number for this question
usol = np.array([[analytical_solution(xi, ti, Re_fixed) for xi in x] for ti in t])

# Save snapshot data
np.savez("dataset/Burgers_snapshots.npz", t=t, x=x, usol=usol)
print(f"Snapshot data generated with m={m}, dt={dt}, dx={dx}.")

Snapshot data generated with m=20, dt=0.01, dx=0.001.


### Question 2.2: Construct ROM and Determine t∗

In [8]:
# Step 1: Perform SVD

# Load snapshot data
data = np.load("dataset/Burgers_snapshots.npz")
t = data["t"]  # Time grid
x = data["x"]  # Spatial grid
usol = data["usol"]  # Snapshot matrix

# Perform SVD on A
A = usol  # Snapshot matrix
U, S, VT = svd(A, full_matrices=False)

# Select k modes based on cumulative energy threshold
alpha_SVD = 0.01  # Error threshold
#k = np.sum(np.cumsum(S**2) / np.sum(S**2) < 1 - alpha_SVD)
k =2
print(f"Number of selected modes (k): {k}")

# Extract the first k modes
U_k = U[:, :k]  # Temporal coefficients
S_k = S[:k]     # Singular values
V_k = VT[:k, :]  # Spatial modes

# Step 2: Fit uq(ti) Using GPR

# Fit GPR models for each mode q=1,...,k
gp_models = []
for q in range(k):
    kernel = C(1.0, (1e-4, 1e4)) * RBF(1.0, (1e-4, 1e4))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-6)
    gp.fit(t.reshape(-1, 1), U_k[:, q])  # Fit GPR for temporal mode u_q(t)
    gp_models.append(gp)

# Step 3: Predict ROM Solution for New Time Points

# Predict ROM solution u_ROM(t, x)
def predict_u_rom(t_new, x, gp_models, S_k, V_k):
    U_pred = np.zeros((len(t_new), k))
    for q, gp in enumerate(gp_models):
        U_pred[:, q] = gp.predict(t_new.reshape(-1, 1))
    
    A_pred = np.dot(U_pred * S_k, V_k)  # A ≈ u_ROM(t, x)
    return A_pred

# Example: Predict u_ROM(t, x) for a new time range
t_new = np.arange(0, 2.0, dt)  # New time range
u_rom_pred = predict_u_rom(t_new, x, gp_models, S_k, V_k)

# Step 4: Determine Maximum Permissible Forecast Time t∗

# Compute average standard deviation σ̄(t')
def compute_avg_std(t_new, gp_models):
    std_total = np.zeros(len(t_new))
    for gp in gp_models:
        _, std = gp.predict(t_new.reshape(-1, 1), return_std=True)
        std_total += std
    return std_total / len(gp_models)

# Determine t* based on σ̄(t') ≤ σ_tolerance
sigma_tolerance = 0.1  # Allowed prediction error threshold
std_avg = compute_avg_std(t_new, gp_models)

# Find the last time index where std_avg is below the threshold
t_star_index = np.where(std_avg <= sigma_tolerance)[0][-1]
t_star = t_new[t_star_index]
print(f"Maximum permissible forecast time t^*: {t_star}")


Number of selected modes (k): 2
Maximum permissible forecast time t^*: 0.66


### Question 2.3: Predict Full-Order Solutions Up to t∗

In [9]:
# Use the ROM to predict up to t*
u_rom_at_t_star = predict_u_rom(np.array([t_star]), x, gp_models, S_k, V_k)

# Take u_rom_at_t_star as initial condition for next step
u_full_t_star = u_rom_at_t_star  # Full-order solution at t*
print(f"Full-order solution at t^* = {t_star} is ready.")

Full-order solution at t^* = 0.66 is ready.


### Question 2.4: Repeat Steps 1-3 Until T=2.0

In [10]:
T_target = 2.0  # Target time
t_current = t  # Current time grid
u_rom_current = usol  # Initial solution

while np.max(t_current) < T_target:
    # Recompute SVD and GPR models
    A = u_rom_current  # Update A matrix
    U, S, VT = svd(A, full_matrices=False)
    k = np.sum(np.cumsum(S**2) / np.sum(S**2) < 1 - alpha_SVD)
    U_k = U[:, :k]
    S_k = S[:k]
    V_k = VT[:k, :]
    
    gp_models = []
    for q in range(k):
        kernel = C(1.0, (1e-4, 1e4)) * RBF(1.0, (1e-4, 1e4))
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-6)
        gp.fit(t_current.reshape(-1, 1), U_k[:, q])
        gp_models.append(gp)
    
    # Predict ROM solution for the new time range
    t_new = np.linspace(np.max(t_current), np.max(t_current) + 0.5, 50)
    u_rom_pred = predict_u_rom(t_new, x, gp_models, S_k, V_k)
    
    # Update current time and solution
    t_current = np.concatenate([t_current, t_new])
    u_rom_current = np.vstack([u_rom_current, u_rom_pred])
    
    # Check if t^* has reached the target time
    if np.max(t_current) >= T_target:
        break

print(f"Long-time prediction completed up to T = {T_target}.")


Long-time prediction completed up to T = 2.0.
